# Need to figure out how to confirm that the format is correct...

In [1]:
def dup_entries(df):
    duplicates = df[df.duplicated(subset=['racer_id', 'discipline'], keep=False)]
    
    if not duplicates.empty:
        print("Found duplicates for (racer_id, discipline):")
        return duplicates
    else:
        print("No duplicates found.")
        return None

# Create the connection

In [2]:
import pandas as pd
import sqlite3
from helper_functions import prep_race_results, upload_results, drop_nulls

# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


# Upload timetrial data (just race results)

In [3]:
import pandas as pd
import numpy as np
from helper_functions import clean_string, RACE_RESULT_COL_ORDER, upload_results

In [5]:
result_cols = ["bib", "discipline", "racer_id", "run1", "run2"]
df_load = pd.read_csv("2025/241229FnGRaceSimResultsBoth.csv", header=None, names=result_cols)

In [6]:
df_load["racer_id"] = df_load["racer_id"].apply(clean_string)
dup_entries(df_load)
df_load.replace({'DNF': 9998,"DSQ": 9998, "DNS": 9999, pd.NA: 9999}, inplace=True)

No duplicates found.


In [7]:
df_load.head(2)

,bib,discipline,racer_id,run1,run2
0,1,SKI,alexhenry,9999,9999
1,2,SKI,tonypetrella,9999,9999


In [8]:
race_date = '12/29/2024'
race_id = 241229
description = '2025 Race Sim (time trials)'

df_load['run1'] = pd.to_numeric(df_load['run1']) # errors='coerce')
df_load['run2'] = pd.to_numeric(df_load['run2']) # errors='coerce')
df_load['best_time'] = df_load[['run1', 'run2']].min(axis=1)
df_load["points"] = np.nan
df_load["team"] = np.nan
df_load["tier"] = np.nan
df_load["race_id"] = race_id

In [9]:
df_load[RACE_RESULT_COL_ORDER].head(2)

,racer_id,discipline,team,tier,run1,run2,best_time,points,race_id,bib
0,alexhenry,SKI,NaN,NaN,9999.0,9999.0,9999.0,NaN,241229,1
1,tonypetrella,SKI,NaN,NaN,9999.0,9999.0,9999.0,NaN,241229,2


In [12]:
#upload_results(
#    df_load[RACE_RESULT_COL_ORDER],
#    race_id,
#    race_date,
#    description,
#    conn
#)

# Update team data (new year start list)!

In [7]:
import os 

os.getcwd()

'/Users/will/Documents/github/cartar/alpine-fng.github.io/data'

In [12]:
teams_df = pd.read_csv('2025/2025StartList.csv')
teams_df = drop_nulls(teams_df, "name")
teams_df["racer_id"] = teams_df["name"].apply(clean_string)
teams_df.columns = ["bib", "discipline", "name", "tier", "team", "racer_id"]
teams_df["year"] = 2025

In [13]:
teams_df

,bib,discipline,name,tier,team,racer_id,year
0,1,SKI,David Moull,11,Green,davidmoull,2025
1,2,SKI,Shanna Reid,11,Purple,shannareid,2025
2,3,SKI,Devesh Varma,11,Orange,deveshvarma,2025
3,4,SKI,Dave Graff,11,Green,davegraff,2025
4,5,SKI,JD Etter,11,Yellow,jdetter,2025
...,...,...,...,...,...,...,...
95,180,SNBD,Thomas Daly,12,Blue,thomasdaly,2025
96,132,SNBD,Sheri Ramshaw,13,Blue,sheriramshaw,2025
97,104,SNBD,Jacob Max Farber,12,Pink,jacobmaxfarber,2025
98,105,SNBD,Steve Crawford,13,Orange,stevecrawford,2025


In [15]:
teams_df.to_sql('Teams', conn, if_exists='append', index=False)
conn.commit()

## Fix team names:

In [17]:
foo = pd.read_sql_query("SELECT * FROM Teams WHERE team = 'Green' AND year = 2025 ORDER BY tier;", conn)
foo

,bib,discipline,name,tier,team,racer_id,year
0,84,SKI,Mclean Wood,1,Green,mcleanwood,2025
1,74,SKI,William Cartar,2,Green,williamcartar,2025
2,67,SKI,Brad Brock,3,Green,bradbrock,2025
3,63,SKI,Mark Laurie,4,Green,marklaurie,2025
4,57,SKI,Todd Robertson,5,Green,toddrobertson,2025
5,46,SKI,Craig Pitman,6,Green,craigpitman,2025
6,38,SKI,David Eva,7,Green,davideva,2025
7,30,SKI,Sarah Robertson,8,Green,sarahrobertson,2025
8,19,SKI,Audun Breder,9,Green,audunbreder,2025
9,12,SKI,Antonio Liberatore,10,Green,antonioliberatore,2025


In [19]:
sql = """
UPDATE Teams
SET team = 'Orange Crush'
WHERE team = 'Orange'
AND year = 2025;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

In [18]:
sql = """
UPDATE Teams
SET team = 'HULK SMASH (Green)'
WHERE team = 'Green'
AND year = 2025;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

# First race upload 2025

In [3]:
path = '2025/250112FnGResultsBoth.csv'
race_date = '01/12/2025'
race_id = 250112
year = 2025
description = '2025 F&G Race #1'
N_tiers=13
N_teams=8 
max_points=10


In [4]:
results = prep_race_results(
    path,
    race_id,
    year,
    N_tiers,
    N_teams,
    max_points,
    conn
)

/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:140: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  team_df.loc[-1] = [pd.NA, pd.NA, pd.NA, tier, team, pd.NA, pd.NA, pd.NA, avg_points]
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df.loc[-1] = [pd.NA, pd.NA, pd.NA, tier, team, pd.NA, pd.NA, pd.NA, avg_points]
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:140: FutureWarning: The behavior of DataFrame

In [5]:
results.sort_values(by=["tier", "points"], ascending=[True, False], inplace=False).to_csv("test_results.csv")

In [6]:
# upload results:
upload_results(results, race_id, race_date, description, conn)

# Upload second race

# Upload 3rd race

# Upload 4th race

# Upload race data (calc points)
[previous]

In [ ]:
path = 'data/2024/240218FnGResultsBoth.csv'
race_date = '02/18/2024'
race_id = 240218
description = '2024 F&G Race #4'
N_tiers=13
N_teams=8 

""" # Comment out for now
upload_new_race_results(
    path,
    race_date,
    race_id,
    description,
    N_tiers,
    N_teams,
    conn
)
"""